# Face_Recognition Test

# Final Code

In [30]:

# Final code with annotation working but some images not able to process

import cv2
import face_recognition
import pickle
import os
import time
import shutil

import os
from PIL import Image

 
# Function to count the number of image files in a folder

# Path to the folder containing the images

def count_images_with_format(test_images_folder, formats=None):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']
    image_count = 0
    formats_set = set()

    for filename in os.listdir(test_images_folder):
        if os.path.isfile(os.path.join(test_images_folder, filename)):
            file_ext = os.path.splitext(filename)[1].lower()
            if file_ext in image_extensions:
                image_count += 1
                if formats is not None:
                    formats_set.add(file_ext)

    if formats is not None:
        return image_count, formats_set
    else:
        return image_count

'''
# Count total number of image files and check their formats
total_images, image_formats = count_images_with_format(test_images_folder, formats=True)

print("Total number of image files:", total_images)
print("Image formats present:", image_formats)

# Check if all images have the same format
if len(image_formats) == 1:
    print("All images have the same format:", image_formats.pop())
else:
    print("Images have different formats.")
'''

def recognize_faces(image_path, output_folder, processed_folder, error_folder):
    try:
        # Load trained model from pickle file
        pickle_filename = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\trained_models\\face_encodings_custom.pickle"
        with open(pickle_filename, "rb") as f:
            encodings_data = pickle.load(f)

        # Extract encodings and names from the trained model
        encodings = encodings_data["encodings"]
        names = encodings_data["names"]

        total_images_identified = 0
        total_images_processed = 0
        total_errors = 0

        # Load image
        img = cv2.imread(image_path)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Detect faces in the image
        face_locations = face_recognition.face_locations(rgb_img)
        face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

        # Recognize faces
        if len(face_locations) > 0:
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Compare face encoding with the encodings in the trained model
                matches = face_recognition.compare_faces(encodings, face_encoding)
                name = "Unknown"

                # Check for a match
                if True in matches:
                    matched_index = matches.index(True)
                    name = names[matched_index]
                    total_images_identified += 1

                    # Draw rectangle around the face
                    cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)

                    # Put the recognized name on the image
                    cv2.putText(img, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

                    # Create a folder for the identified person if it doesn't exist
                    person_folder = os.path.join(output_folder, name)
                    os.makedirs(person_folder, exist_ok=True)

                    # Save the annotated image to the person's folder
                    output_path = os.path.join(person_folder, os.path.basename(image_path))
                    cv2.imwrite(output_path, img)
                else:
                    # Move the unmatched image to the error folder
                    shutil.move(image_path, os.path.join(error_folder, os.path.basename(image_path)))
                    total_errors += 1
        else:
            # Move the image to the error folder if no face is detected
            shutil.move(image_path, os.path.join(error_folder, os.path.basename(image_path)))
            total_errors += 1

        # Move the original image to the processed folder
        shutil.move(image_path, os.path.join(processed_folder, os.path.basename(image_path)))
        total_images_processed += 1

        return total_images_identified, total_images_processed, total_errors
        
    except FileNotFoundError:
        print(f"File '{image_path}' not found. Skipping...")
        return 0, 0, 1

# Path to the folder containing test images
test_images_folder = 'C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\test\\'

# Paths for result
output_folder = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\result\\output\\"
processed_folder = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\result\\processed\\"
error_folder = "C:\\Users\\Learning\\udemy\\Hackathon\\ultimate\\dataset\\result\\error\\"

os.makedirs(output_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(error_folder, exist_ok=True)

# Measure execution time
start_time = time.time()
print("Execution started...")

# Call the function for each image in the test folder
total_images_identified = 0
total_images_processed = 0
total_errors = 0
total_images_saved = 0

for filename in os.listdir(test_images_folder):
    if filename.endswith('.jpg')  or filename.endswith('.jpeg') or filename.endswith('.png'):
        image_path = os.path.join(test_images_folder, filename)
        identified, processed, errors = recognize_faces(image_path, output_folder, processed_folder, error_folder)
        total_images_identified += identified
        total_images_processed += processed
        total_errors += errors
        if identified > 0:
            total_images_saved += 1

# Calculate execution time
execution_time = time.time() - start_time

# Print statistics
print("===================================")
print("Total execution time: {:.2f} seconds".format(execution_time))
print("Execution ended.")
print("Total images identified: ", total_images_identified)
print("Total images processed: ", total_images_processed)
print("Total images with errors: ", total_errors)
print("Total images annotated in output folder:", total_images_saved)
print("===================================")

print("Execution Completed...")


Execution started...
Total execution time: 0.00 seconds
Execution ended.
Total images identified:  0
Total images processed:  0
Total images with errors:  0
Total images annotated in output folder: 0
Execution Completed...
